# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tsiombe,-25.3000,45.4833,17.52,80,0,2.01,MG,1714966424
1,1,badger,64.8000,-147.5333,14.95,38,100,6.69,US,1714966424
2,2,isafjordur,66.0755,-23.1240,0.79,80,8,5.08,IS,1714966424
3,3,la passe,45.5549,-0.8967,12.77,86,44,5.63,FR,1714966424
4,4,edinburgh of the seven seas,-37.0676,-12.3116,13.74,64,86,14.00,SH,1714966424


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map 
# Utilize hvplot.points in conjuction with our city data dataframe to map each city in the dataset. Apply appropriate -
# - x and y axis labels, titles, hvplot tile, hover cols, sizing bases on humidty, height/width, coloring based on indiviudal cities, etc.
city_data_df.hvplot.points('Lng','Lat', geo=True, tiles='OSM', height=580, width=1000, color='City', size='Humidity',
                           alpha=0.5,
                           title="Global Cities Map: Humidity Sized",
                           xlabel="Longitude",
                           ylabel="Latitude",
                           hover_cols=['Country', 'City', 'Max Temp', 'Humidity'])

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Apply a series of filters that cut down our dataset to align based on provided parameters: -
# Max Temp between 27 and 21 degrees Celsius, Wind Speed less than 4.5 m/s and zero Cloudiness
temp_filtered_df = city_data_df.loc[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21)]
wind_filtered_df = temp_filtered_df.loc[temp_filtered_df['Wind Speed'] < 4.5]
final_cloud_filtered_df = wind_filtered_df.loc[wind_filtered_df['Cloudiness'] == 0]

# Drop any rows with null values
# Create a new dataframe to store our data after null values have been dropped
final_dataframe = final_cloud_filtered_df.dropna()

# Display sample data
final_dataframe

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
123,123,sao miguel do iguacu,-25.3481,-54.2378,23.69,45,0,3.63,BR,1714966436
182,182,morehead city,34.7229,-76.7260,23.00,86,0,4.12,US,1714966332
188,188,bataipora,-22.2953,-53.2711,21.37,44,0,2.73,BR,1714966443
216,216,boda,4.3189,17.4695,23.01,70,0,0.65,CF,1714966446
224,224,camapua,-19.5314,-54.0439,21.85,55,0,2.76,BR,1714966447
241,241,poxoreo,-15.8372,-54.3892,21.08,70,0,1.88,BR,1714966448
250,250,mount pleasant,32.7941,-79.8626,23.33,91,0,2.57,US,1714966449
295,295,san ignacio de velasco,-16.3667,-60.9500,23.09,66,0,1.93,BO,1714966454
313,313,sao joao da barra,-21.6403,-41.0511,24.84,84,0,4.27,BR,1714966212
333,333,tokar,18.4254,37.7290,26.04,63,0,1.07,SD,1714966457


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Column filter and copy our already narrowed down dataframe to a new dataframe with required column structure
hotel_df = final_dataframe[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# Create this new "Hotel Name" column and store nothing within its values for the moment - our loop will populate values momentarily
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
123,sao miguel do iguacu,BR,-25.3481,-54.2378,45,
182,morehead city,US,34.7229,-76.7260,86,
188,bataipora,BR,-22.2953,-53.2711,44,
216,boda,CF,4.3189,17.4695,70,
224,camapua,BR,-19.5314,-54.0439,55,
241,poxoreo,BR,-15.8372,-54.3892,70,
250,mount pleasant,US,32.7941,-79.8626,91,
295,san ignacio de velasco,BO,-16.3667,-60.9500,66,
313,sao joao da barra,BR,-21.6403,-41.0511,84,
333,tokar,SD,18.4254,37.7290,63,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
# Setting radius to 5000m to encompass a total area of 10,000m around each coordinates
# Include proper parameter values within params dictionary according to Geoapify API documentation. This will include: - 
# - category, API key & radius before adding filter and bias criteria within API call loop that corresponds to specific coordinates
radius = 5000
category = "accommodation.hotel"
params = {
    "apiKey": geoapify_key,
    "radius": radius,
    "categories": category  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # Reference "row" (iterator) and use .loc in order to obtain a single/specific value from latitude and longitude columns - 
    # - within our dataframe
    lat = row.loc['Lat']
    lon = row.loc['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    # As mentioned above these additional parameters are location/coordinates specific and as such must be included in API call loop
    # proper f string format was confirmed via Geoapify API documentation
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results and display/print accordingly
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sao miguel do iguacu - nearest hotel: No hotel found
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham
bataipora - nearest hotel: No hotel found
boda - nearest hotel: No hotel found
camapua - nearest hotel: No hotel found
poxoreo - nearest hotel: No hotel found
mount pleasant - nearest hotel: Old Village Post House
san ignacio de velasco - nearest hotel: Hotel Caparu
sao joao da barra - nearest hotel: Pousada Mediterrâneo
tokar - nearest hotel: No hotel found
sedhiou - nearest hotel: Hôtel Chez Nous
huayin - nearest hotel: No hotel found
pirane - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
123,sao miguel do iguacu,BR,-25.3481,-54.2378,45,No hotel found
182,morehead city,US,34.7229,-76.7260,86,"Bask Hotel Big Rock Landing, Trademark Collect..."
188,bataipora,BR,-22.2953,-53.2711,44,No hotel found
216,boda,CF,4.3189,17.4695,70,No hotel found
224,camapua,BR,-19.5314,-54.0439,55,No hotel found
241,poxoreo,BR,-15.8372,-54.3892,70,No hotel found
250,mount pleasant,US,32.7941,-79.8626,91,Old Village Post House
295,san ignacio de velasco,BO,-16.3667,-60.9500,66,Hotel Caparu
313,sao joao da barra,BR,-21.6403,-41.0511,84,Pousada Mediterrâneo
333,tokar,SD,18.4254,37.7290,63,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Utilize hvplot.points in conjuction with our updated hotel_df dataframe to map each city in the dataset. Apply appropriate -
# - x and y axis labels, titles, hvplot tile, hover cols (INCLUDING HOTEL NAME), sizing bases on humidty, - 
# - height/width, coloring based on indiviudal cities, etc.
# Configure the map plot/display
hotel_df.hvplot.points('Lng','Lat', geo=True, tiles='OSM', height=580, width=1000, color='City', size='Humidity',
                           alpha=0.5,
                           title="Cities With Ideal Weather: Humidity Sized & Hotel Name Included",
                           xlabel="Longitude",
                           ylabel="Latitude",
                           hover_cols=['Country', 'City', 'Max Temp', 'Humidity', 'Hotel Name'])

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)